In [ ]:
import shutil

extraction_path = '/content/drive/MyDrive/Abdelrahman_s/new_new'

try:
    shutil.rmtree(extraction_path)  # This will delete the directory and its contents
    print(f"Deleted {extraction_path}")
except FileNotFoundError:
    print(f"{extraction_path} not found")
except PermissionError:
    print(f"Permission denied to delete {extraction_path}")
except Exception as e:
    print(f"An error occurred: {str(e)}")


Deleted /content/drive/MyDrive/Abdelrahman_s/new_new


In [ ]:
import zipfile

zip_file_path = '/content/drive/MyDrive/Abdelrahman_s/new.zip'
extraction_path = '/content/drive/MyDrive/Abdelrahman_s/new_new/images'

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
import pandas as pd
import os


In [ ]:
train_csv_path = '/content/drive/MyDrive/Abdelrahman_s/train.csv'
valid_csv_path = '/content/drive/MyDrive/Abdelrahman_s/valid.csv'
test_csv_path = '/content/drive/MyDrive/Abdelrahman_s/test.csv'
image_folder = '/content/drive/MyDrive/Abdelrahman_s/new_new/images/new_new_images'

train_df = pd.read_csv(train_csv_path)
valid_df = pd.read_csv(valid_csv_path)
test_df = pd.read_csv(test_csv_path)

img_width = int(train_df['width'].values[0])
img_height = int(train_df['height'].values[0])

BATCH_SIZE = 32
EPOCHS = 200
num_classes = len(train_df['class'].unique())


In [ ]:
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

def image_exists(image_path):
    return os.path.exists(os.path.join(image_folder, image_path))

train_df = train_df[train_df['filename'].apply(image_exists)]
valid_df = valid_df[valid_df['filename'].apply(image_exists)]
test_df = test_df[test_df['filename'].apply(image_exists)]

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

# Add TensorBoard callback
log_dir = '/content/drive/MyDrive/Abdelrahman_s/logsvgg16'  # Adjust the directory as needed
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images=True)

model_checkpoint = ModelCheckpoint('best_model_vgg16.h5', save_best_only=True)
early_stopping = EarlyStopping(patience=10)

train_generator = datagen.flow_from_dataframe(
    train_df,
    directory=image_folder,
    x_col="filename",
    y_col="class",
    target_size=(img_width, img_height),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
)

valid_generator = datagen.flow_from_dataframe(
    valid_df,
    directory=image_folder,
    x_col="filename",
    y_col="class",
    target_size=(img_width, img_height),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // BATCH_SIZE,
    callbacks=[model_checkpoint, early_stopping, tensorboard_callback]  # Include TensorBoard callback
)

test_generator = datagen.flow_from_dataframe(
    test_df,
    directory=image_folder,
    x_col="filename",
    y_col="class",
    target_size=(img_width, img_height),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
)

test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy}")

model.load_weights('best_model_vgg16.h5')
model.save("vgg16_image_classification_model.h5")


Found 22352 validated image filenames belonging to 2 classes.
Found 1242 validated image filenames belonging to 2 classes.
Epoch 1/200
698/698 [==============================] - 248s 354ms/step - loss: 0.6298 - accuracy: 0.6468 - val_loss: 0.5883 - val_accuracy: 0.6521
Epoch 2/200
698/698 [==============================] - 244s 349ms/step - loss: 0.5931 - accuracy: 0.6779 - val_loss: 0.5723 - val_accuracy: 0.6785
Epoch 3/200
698/698 [==============================] - 244s 349ms/step - loss: 0.5815 - accuracy: 0.6871 - val_loss: 0.5647 - val_accuracy: 0.6982
Epoch 4/200
698/698 [==============================] - 244s 349ms/step - loss: 0.5794 - accuracy: 0.6882 - val_loss: 0.5630 - val_accuracy: 0.7155
Epoch 5/200
698/698 [==============================] - 235s 336ms/step - loss: 0.5772 - accuracy: 0.6931 - val_loss: 0.5597 - val_accuracy: 0.7130
Epoch 6/200
698/698 [==============================] - 195s 279ms/step - loss: 0.5731 - accuracy: 0.6927 - val_loss: 0.5451 - val_accuracy: 0.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/Abdelrahman_s/logsvgg16


Output hidden; open in https://colab.research.google.com to view.